In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt
from textwrap import wrap
from tqdm import tqdm

image_path = '/Users/scotttow123/Documents/BYUI/Senior Data Science Project/Images'
caption_file = "/Users/scotttow123/Documents/BYUI/Senior Data Science Project/captions.txt"

try:
    data = pd.read_csv(caption_file)
    print("Data loaded successfully")
except FileNotFoundError:
    print(f"Error: The file {caption_file} was not found.")
    data = pd.DataFrame() 

data.head()

In [ ]:
def read_image(path, img_size=224):
    """Reads and preprocesses an image."""
    try:
        img = load_img(path, color_mode='rgb', target_size=(img_size, img_size))
        img = img_to_array(img)
        img = img / 255.0
        return img
    except Exception as e:
        print(f"Error reading image {path}: {e}")
        return None

def display_images(temp_df, img_dir, num_images=9):
    """Displays a grid of images with their captions."""
    temp_df = temp_df.reset_index(drop=True)
    plt.figure(figsize=(15, 15))
    for i in range(num_images):
        plt.subplot(3, 3, i + 1)
        image_path = os.path.join(img_dir, temp_df.image[i])
        
        image = read_image(image_path)
        if image is not None:
            plt.imshow(image)
            plt.title("\n".join(wrap(temp_df.caption[i], 20)))
        else:
            plt.imshow(np.ones((224, 224, 3))) 
            plt.title("Image not found")
        plt.axis("off")
    plt.tight_layout()
    plt.show()

display_images(data.sample(9), image_path)

In [ ]:
def preprocess_image(img_path, img_size=224):
    """Loads and preprocesses an image for the VGG16 model."""
    img = load_img(img_path, target_size=(img_size, img_size))
    img = img_to_array(img)
    img = preprocess_input(img) 
    return img

def extract_features_batch(image_dir, model, img_size=224, batch_size=32):
    """Extract features from images using a pre-trained model in batches."""
    features = {}
    image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir)]
    
    for i in tqdm(range(0, len(image_paths), batch_size), desc="Extracting features"):
        batch_paths = image_paths[i:i + batch_size]
        batch_images = []

        for img_path in batch_paths:
            img = preprocess_image(img_path, img_size)
            batch_images.append(img)

        batch_images = np.array(batch_images)
        batch_features = model.predict(batch_images, verbose=0)

        for img_path, feature in zip(batch_paths, batch_features):
            img_name = os.path.basename(img_path)
            features[img_name] = feature.flatten()

    return features


def display_images(temp_df, img_dir, num_images=9):
    """Displays a grid of images with their captions."""
    temp_df = temp_df.reset_index(drop=True)
    plt.figure(figsize=(15, 15))
    for i in range(num_images):
        plt.subplot(3, 3, i + 1)
        image_path = os.path.join(img_dir, temp_df.image[i])
        image = preprocess_image(image_path, img_size=224) 
        plt.imshow(image / 255.0)  
        plt.title("\n".join(wrap(temp_df.caption[i], 20)))
        plt.axis("off")
    plt.tight_layout()
    plt.show()

# Display 9 random images with captions
display_images(data.sample(9), image_path)

In [ ]:
def display_caption_stats(captions):
    """Displays some statistics about the captions."""
    caption_lengths = captions.apply(lambda x: len(x.split()))
    print(f"Total captions: {len(captions)}")
    print(f"Average caption length: {caption_lengths.mean():.2f} words")
    print(f"Max caption length: {caption_lengths.max()} words")
    print(f"Min caption length: {caption_lengths.min()} words")

display_caption_stats(data['caption'])

display_images(data.sample(9), image_path)